# Reconstructing the path
In this assignment you will use Spark to compute the shortest path between two vertices. In the video, you have learned how to compute the distances between a source vertex and all other vertices in a graph. Now, your task is to reconstruct the shortest path, that is a sequence of vertices connected by the edges.

Dataset location: /data/twitter/twitter_sample_small.txt

Format: user_id \t follower_id

You can start with the code described in "Starter..." (see the next self-reading).

Your task is to find the shortest path between vertices 12 and 34. In case of multiple shortest paths (that is, disjoint paths with the same length), any will suffice. Output format is the sequence of vertices, delimited by a comma, without spaces. For example, the path “12 -> 42 -> 34” should be printed as:

<code>12,42,34</code>

Hint: before submitting, check your stopping criteria. In BFS, the search was exhaustive, and in this task your program may terminate earlier, thus saving some precious time.

The result on the sample dataset:

<code>12,422,53,52,107,20,23,274,34</code>


In [2]:
from pyspark import SparkConf, SparkContext
sc = SparkContext(conf=SparkConf().setAppName("MyApp").setMaster("local"))

In [8]:
def parse_edge(s):
    user, follower = s.split("\t")
    return (int(user), int(follower))

def step(item):
    print(item)
    prev_v, prev_p, next_v = item[0], item[1][0], item[1][1]
    return (next_v, prev_p + [next_v])

def complete(item):
    v, old_p, new_p = item[0], item[1][0], item[1][1]
    return (v, old_p if old_p is not None else new_p)

n = 4 #00 number of partitions
forward_edges = sc.textFile("/data/twitter/twitter_sample_small.txt") \
    .map(parse_edge) \
    .map(lambda x: (x[1], x[0])) \
    .partitionBy(n) \
    .persist()

start_v, end_v = 12 , 34
paths = sc.parallelize([(start_v, [start_v])]).partitionBy(n)

while True:
    candidates = paths.join(forward_edges, n).map(step).reduceByKey(min)
    paths = paths.fullOuterJoin(candidates, n).map(complete, True).persist()
    final_path = paths.lookup(end_v)
    if final_path:
        print ','.join(map(str,final_path[0]))
        break

12,422,53,52,107,20,23,274,34
